# House Prices - v2

In this notebook, I will be using the "House prices" data from Kaggle to create a model that will help to preduct the missing house prices. Version 2 is derived from version 1. The code was shortened by combining operations or using pre-defined functions instead of doing some operations manually.

In the following step, necessary libraries are imported and the data is loaded. 

In [1]:
#imports
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
'''
As can be noted, there are more imports in v2 compared to v1. For example: OneHotEncoder and SimpleImputer.
This is because the functions I will be using to make the code shorter are contained in these libraries. 
'''
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

#loading the data

url_train = "https://raw.githubusercontent.com/Sovima/House-prices/main/train.csv"
url_test = "https://raw.githubusercontent.com/Sovima/House-prices/main/test.csv"

house_data = pd.read_csv(url_train, error_bad_lines=False)
test_data = pd.read_csv(url_test, error_bad_lines=False)


There are a few custom functions that I will be using during process to investigate the data, make transformations more efficient, and/or create the model. You can find them in the cell below

In [2]:

def missing_mean(data, columns):
    """
    Takes the data in column and replaces them with the random values 
    within 1 std of the mean of the column
    """
    mean = data[columns].mean()
    std = data[columns].std()
    is_null = data[columns].isnull().sum()
    rand_mean = np.random.randint(mean-std, mean+std, size = is_null)
    house_dummies_slice = data[columns].copy()
    house_dummies_slice[np.isnan(house_dummies_slice)] = rand_mean
    data[columns] = house_dummies_slice
    data[columns] = data[columns].astype(float)
    
def export_as_csv(predictions):
    """
    Exports the predictions dataset as Predictions_house.csv file in the file_dir 
    """
    import os
    file_dir = "Documents/Sofya/data s. corona/house-prices-advanced-regression-techniques"
    os.chdir(file_dir)
    predictions.to_csv("Predictions_house", index = False)

def forest_model(house_dummies, SalePrice, test_dummies):
    '''
    Takes in house_dummies, test_dummies, and SalePrice to return the Y_predicted sale prices. 
    The Y-predicted is obtained by fitting a Random Forest model with house_dummies and SalePrice 
    and using test_dummies to predict the new prices
    '''
    X_train = house_dummies
    X_val = test_dummies
    Y_train =  SalePrice

    random_forest = RandomForestClassifier(n_estimators=100)
    random_forest.fit(X_train, Y_train)
    Y_prediction = pd.DataFrame(random_forest.predict(X_val))
    return Y_prediction


def models(X, y):
    '''
    Creates different models and evaluates their efficiency through mean_absolute_error
    '''
    X_train, X_val, Y_train, Y_val = train_test_split(X, 
                                                      y, 
                                                      random_state = 0)
    #SGD Classifier
    sgd = linear_model.SGDClassifier(max_iter=5, tol=None)
    sgd.fit(X_train, Y_train)
    Y_pred = sgd.predict(X_val)
    acc_sgd = mean_absolute_error(Y_val, Y_pred)
    #Random Forest
    random_forest = RandomForestClassifier(n_estimators=100)
    random_forest.fit(X_train, Y_train)
    Y_pred = random_forest.predict(X_val)
    acc_random_forest = mean_absolute_error(Y_val, Y_pred)
    #Decision Tree
    d_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
    d_tree.fit(X_train, Y_train)
    Y_pred = d_tree.predict(X_val)
    acc_decision_tree = mean_absolute_error(Y_val, Y_pred)
    
    #Regression
    regr = linear_model.LinearRegression()

    regr.fit(X_train, Y_train)

    Y_pred = regr.predict(X_val)
    acc_linear_reg = mean_absolute_error(Y_val, Y_pred)
    
    return acc_sgd, acc_random_forest, acc_linear_reg, acc_decision_tree


def missing_values(house_data):
    '''
    counts number of missing values in each column and displays a table with the following columns:
    1st column contains column names
    2nd column contains the total number of missing values in that column
    3rd row - percent of missing values of the total number of values in the columns
    '''
    total = house_data.isnull().sum().sort_values(ascending=False)
    percent_1 = house_data.isnull().sum()/house_data.isnull().count()*100
    percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
    return missing_data.head((missing_data['Total'] > 0).sum())

In [3]:
house_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


As can be seen from the first rows of house_data, there are columns with missing values. I will next apply the missing_values function to have a better understanding of all missing values and possibly understand why they are missing

In [4]:
missing_values(house_data)

,Total,%
PoolQC,1453,99.5
MiscFeature,1406,96.3
Alley,1369,93.8
Fence,1179,80.8
FireplaceQu,690,47.3
LotFrontage,259,17.7
GarageCond,81,5.5
GarageType,81,5.5
GarageYrBlt,81,5.5
GarageFinish,81,5.5


From the available data description that is contained in a separate text file we can understand the reason behind such high percentages of missing values in some columns

In [5]:
f = open("/Users/sofyamalashchenko/Documents/Sofya/data s. corona/"
         "house-prices-advanced-regression-techniques/data_description.txt", "r")
print(f.read())
f.close()

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

In columns "PoolQC", "MiscFeature", "Alley", and "Fence" empty cell in most cases(there is still a chance of the value actually missing), means that the house does not have the feature described in the column. This means that those cells should be treated as text(for example "none") rather than a missing value. 

To do that I will separate text data into several groups: columns with text that can have a numerical meaning, columns where text cannot be numerically converted, and columns with a high number of unique entries that do not have a numerical meaning(for example, street name)

In this model, I decided not to include the 3rd type of data into the final calculations. The rest will be kept and converted. I will start with LabelEncoder which will automatically assign numerical values to the text values in both train and test datasets. 

In [6]:
label_encoder_cols = [
    'LotShape', "Street", "MasVnrType", 'LandContour', 'Utilities',
    'LandSlope', 'BldgType', 'HouseStyle', 'ExterQual', 'ExterCond',
    'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
    'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
    'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond',
    'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature']

my_encoder = LabelEncoder()

#The following function makes sure unique entries in house_data and test_data are the same
good_encoder_cols = [col for col in label_encoder_cols if 
                   set(house_data[col]) >= set(test_data[col])]

label_house_data = house_data[good_encoder_cols].copy()
label_test_data = test_data[good_encoder_cols].copy()
label_house_data.loc[:,"Electrical"] = (label_house_data.fillna('SBrkr'))["Electrical"]
label_test_data.loc[:,"Electrical"] = (label_test_data.fillna("SBrkr"))["Electrical"]
label_house_data[good_encoder_cols] = label_house_data[good_encoder_cols].fillna("none")
label_test_data[good_encoder_cols] = label_test_data[good_encoder_cols].fillna("none")   

#out goes a set of data
for i in good_encoder_cols:
    label_house_data[i] = my_encoder.fit_transform(label_house_data[i])
    label_test_data[i] = my_encoder.transform(label_test_data[i])

After that, I am using OneHotEncoder which is used to encode 2nd type of text values. Each new column refers to a unique entry in the old column

In [7]:
get_dummies_cols = set(label_encoder_cols)-set(good_encoder_cols)
get_dummies_cols
test_data[get_dummies_cols]
get_dummies_cols = set(label_encoder_cols)-set(good_encoder_cols)


house_encoder = (house_data[get_dummies_cols]).fillna(0).copy()
test_encoder = (test_data[get_dummies_cols]).fillna(0).copy()
missing_values(test_encoder)

,Total,%


In [8]:
get_dummies_cols = set(label_encoder_cols)-set(good_encoder_cols)

house_encoder = (house_data[get_dummies_cols]).fillna(0).copy()
test_encoder = (test_data[get_dummies_cols]).fillna(0).copy()

my_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)


house_encoder = pd.DataFrame(my_encoder.fit_transform(house_encoder))
test_encoder = pd.DataFrame(my_encoder.transform(test_encoder))

#out goes a set of data
house_encoder.columns = my_encoder.get_feature_names()
test_encoder.columns = my_encoder.get_feature_names()

After that I will be looking at int and float columns that have missing values. My decision was to replace those values using the SimpleImputer which has a default strategy parameter = mean

In [9]:
drop_columns = ['GarageYrBlt', 'SalePrice']

num_columns = (set((house_data.dtypes == int).index[house_data.dtypes == int]) | 
               set((house_data.dtypes == float).index[house_data.dtypes == float])) - set(drop_columns)
numerical_house = house_data[num_columns].copy()
numerical_test = test_data[num_columns].copy()

num_imputer = SimpleImputer()


#out goes a set of data
numerical_house_imp = pd.DataFrame(num_imputer.fit_transform(numerical_house), columns = num_columns)
numerical_test_imp = pd.DataFrame(num_imputer.transform(numerical_test), columns = num_columns)


In [10]:
print('Number of object columns is {}'.format((house_data.dtypes == 'object').sum()))
print('Number of integer columns is {}'.format((house_data.dtypes == 'int').sum()))
print('Number of float columns is {}'.format((house_data.dtypes == 'float').sum()))
print('Total number of columns is {}'.format(len(house_data.columns)))

Number of object columns is 43
Number of integer columns is 35
Number of float columns is 3
Total number of columns is 81


This means that the data in house_data is stored in 3 differen types: integer, float, and object.
The models are using numerical values to build formulas. This is why working with integers and floats is straightforward while objects must be converted to numerical values before using them within to fit a model. 

In [11]:
house_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Before looking at the models, I need to connect the data and find the columns with acceptable correlation

In [12]:
"""
Connecting the Data----------------------------------------------------------------------------------------
"""
#Know and understand the concat notation!!
concat_house = pd.concat([label_house_data, house_encoder], axis=1)
concat_test = pd.concat([label_test_data, test_encoder], axis=1)

concat_house = pd.concat([concat_house, numerical_house_imp], axis=1)
concat_test = pd.concat([concat_test, numerical_test_imp], axis=1)


y_train = house_data["SalePrice"]

corr = pd.DataFrame(concat_house.corrwith(y_train))
'''
Finding columns with acceptable correlation-----------------------------------------------------------------
'''

#Following is an important function that allows to select df columns based on a condition
#Should be memorized!

high_corr_cols = set(corr.index[corr[0] > 0.2]) | set(corr.index[corr[0] < -0.2])

#Note that sets can be used to select columns of a dataframe
high_corr_house = concat_house[high_corr_cols]
high_corr_test = concat_test[high_corr_cols]

Now that all columns are in numerical format and are connected, I can start looking at which model will present the best given this dataset. I will be testing 4 models: RandomForest, RandomTree, Regression, and SGD classifier. I am sure there are many ways to check model's efficiency, however I am currently only familiar with mean_absolute_error. 

In [13]:
models(high_corr_house, y_train)

(59529.4, 24167.69589041096, 21660.60403825978, 45763.334246575345)

We can see that RandomForest and DecisionTree have performed better than the other 2 models. Since RandomForest and DecisionTree have close results, I will use the first one because it provides a higher degree of randomness.

In [14]:
predictions = pd.DataFrame(test_data["Id"]).copy()
predictions["SalePrice"] = pd.DataFrame(forest_model(high_corr_house, y_train, high_corr_test))
